In [ ]:
import pandas as pd
import datetime
import os
import pytz
utc = pytz.utc

dataFileColumns = ["timestamp", "stage", "endTimestamp"]
outputFileName = "appleWatch6pillowSleepStages.parquet.gzip"

#get list of files in current directroy
files = [f for f in os.listdir("./") if os.path.isfile(f)]

#if the output file isn't already made, make it
if not (outputFileName in files):
    print("making new df")
    emptydf = pd.DataFrame(columns = dataFileColumns)
    emptydf.set_index('timestamp')
    print(emptydf)
        
    #save as a parquet file
    emptydf.to_parquet(outputFileName, compression='gzip') 


df = pd.read_parquet(outputFileName)

print(df.head(100))

In [ ]:
#import it as one big string
#split at ZSLEEPSTAGEDATAPOINT 
#then split at ZSNOOZELAB

#each line will be it's own stage split at Z

#i'm going to add each line to a temporary DF 
#sort by timestamp and reindex
#iterate over indexes calculate the time in the sleep stage coalesing neighboring sleep stages
#save new rows to the proper df to be saved